For this lab and in the next lessons we will build a model on customer churn binary classification problem. You will be using files_for_lab/Customer-Churn.csv file.

Scenario:

You are working as an analyst with this internet service provider. You are provided with this historical data about your company's customers and their churn trends. Your task is to build a machine learning model that will help the company identify customers that are more likely to default/churn and thus prevent losses from such customers.

Instructions
In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned on the class.

Here is the list of steps to be followed (building a simple model without balancing the data):

Import the required libraries and modules that you would need.
Read that data into Python and call the dataframe churnData.
Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.
Check for null values in the dataframe. Replace the null values.
Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:
Scale the features either by using normalizer or a standard scaler.
Split the data into a training set and a test set.
Fit a logistic regression model on the training data.
Check the accuracy on the test data.
Note: So far we have not balanced the data.

Managing imbalance in the dataset

Check for the imbalance.
Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
Each time fit the model and see how the accuracy of the model is.

In [37]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score

In [13]:
root = pd.read_csv(r'C:\Users\aidag\OneDrive\Escritorio\ironhack\lab-handling-data-imbalance-classification\files_for_lab\Customer-Churn.csv')
df_copy = root.copy()
df_copy.head(5)

gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  Female              0     Yes         No       1           No   
1    Male              0      No         No      34          Yes   
2    Male              0      No         No       2          Yes   
3    Male              0      No         No      45           No   
4  Female              0      No         No       2          Yes   

  OnlineSecurity OnlineBackup DeviceProtection TechSupport StreamingTV  \
0             No          Yes               No          No          No   
1            Yes           No              Yes          No          No   
2            Yes          Yes               No          No          No   
3            Yes           No              Yes         Yes          No   
4             No           No               No          No          No   

  StreamingMovies        Contract  MonthlyCharges TotalCharges Churn  
0              No  Month-to-month           29.85        29.85    No  
1              No        One year           56.95       1889.5    No  
2              No  Month-to-month           53.85       108.15   Yes  
3              No        One year           42.30      1840.75    No  
4              No  Month-to-month           70.70       151.65   Yes

In [14]:
df_copy.columns = df_copy.columns.str.capitalize()

In [15]:
df_copy.dtypes

Gender               object
Seniorcitizen         int64
Partner              object
Dependents           object
Tenure                int64
Phoneservice         object
Onlinesecurity       object
Onlinebackup         object
Deviceprotection     object
Techsupport          object
Streamingtv          object
Streamingmovies      object
Contract             object
Monthlycharges      float64
Totalcharges         object
Churn                object
dtype: object

In [39]:
df_copy['Contract'].value_counts()

Contract
Month-to-month    3875
Two year          1685
One year          1472
Name: count, dtype: int64

In [16]:
contract_mapping  = df_copy['Contract'].value_counts()

In [17]:
contract_mapping = {'Month-to-month': 0, 'One year': 1, 'Two year': 2}
df_copy['contract_ordinal'] = df_copy['Contract'].map(contract_mapping)

numerical_data = df_copy.select_dtypes(include=['int64', 'float64'])
categorical_data = df_copy.select_dtypes(include='object')

In [18]:
df_copy['Totalcharges'] = pd.to_numeric(df_copy['Totalcharges'], errors='coerce')

In [19]:
df_copy.isna().sum()

Gender               0
Seniorcitizen        0
Partner              0
Dependents           0
Tenure               0
Phoneservice         0
Onlinesecurity       0
Onlinebackup         0
Deviceprotection     0
Techsupport          0
Streamingtv          0
Streamingmovies      0
Contract             0
Monthlycharges       0
Totalcharges        11
Churn                0
contract_ordinal     0
dtype: int64

In [20]:
df_copy = df_copy.dropna(subset=["Totalcharges"])

In [21]:
df_copy.describe()

Seniorcitizen       Tenure  Monthlycharges  Totalcharges  \
count    7032.000000  7032.000000     7032.000000   7032.000000   
mean        0.162400    32.421786       64.798208   2283.300441   
std         0.368844    24.545260       30.085974   2266.771362   
min         0.000000     1.000000       18.250000     18.800000   
25%         0.000000     9.000000       35.587500    401.450000   
50%         0.000000    29.000000       70.350000   1397.475000   
75%         0.000000    55.000000       89.862500   3794.737500   
max         1.000000    72.000000      118.750000   8684.800000   

       contract_ordinal  
count       7032.000000  
mean           0.688567  
std            0.832934  
min            0.000000  
25%            0.000000  
50%            0.000000  
75%            1.000000  
max            2.000000

 En este punto, estás dividiendo tu conjunto de datos en características (X) y la variable objetivo (y). Luego, divides estos conjuntos en datos de entrenamiento y datos de prueba utilizando la función train_test_split(). Después, aplicas el submuestreo (downsampling) utilizando RandomUnderSampler() para equilibrar las clases en los datos de entrenamiento. Este paso es importante para abordar el desequilibrio de clases en tu conjunto de datos, lo que puede mejorar el rendimiento del modelo de regresión logística al evitar que el modelo se sesgue hacia la clase mayoritaria.

In [31]:
X = df_copy[['Tenure', 'Seniorcitizen', 'Monthlycharges', 'Totalcharges']]
y = df_copy['Churn']

y = df_copy['Churn']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

under_sampler = RandomUnderSampler()
X_resampled_train, y_resampled_train = under_sampler.fit_resample(X_train, y_train)

Punto 2: Aquí estás aplicando el sobre muestreo (upsampling) utilizando RandomOverSampler() para aumentar la cantidad de muestras en la clase minoritaria en los datos de entrenamiento. Esto se hace para equilibrar las clases y evitar el sesgo del modelo hacia la clase mayoritaria.

In [35]:
over_sampler = RandomOverSampler()
X_resampled_over, y_resampled_over = over_sampler.fit_resample(X_train, y_train)

Punto 3: Similar al punto 1, este punto también implica aplicar el submuestreo (downsampling) utilizando RandomUnderSampler() para equilibrar las clases en los datos de entrenamiento. Es una repetición del punto 1, pero podrías estar probando diferentes técnicas de remuestreo para ver cuál funciona mejor para tu conjunto de datos específico.

In [36]:
under_sampler = RandomUnderSampler()
X_resampled_under, y_resampled_under = under_sampler.fit_resample(X_train, y_train)

Modelo de regresión logística:

In [38]:
model = LogisticRegression()
model.fit(X_resampled_train, y_resampled_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Precission :", accuracy)

Precission : 0.7093105899076049
